In [1]:
import os

import dotenv

from time import time, sleep
from algosdk.future import transaction
from algosdk.logic import get_application_address

from store.operations import StoringPool
from bidding.operations import create_bidding_app, place_bid, cancel_bid, accept_bid, close_bidding
from trading.operations import create_trading_app, place_trade, cancel_trade, accept_trade, close_trading
from utils import *
from account import Account

In [2]:
dotenv.load_dotenv('.env')

client = get_algod_client(os.environ.get('ALGOD_URL'), os.environ.get('ALGOD_TOKEN'))

In [3]:
creator = Account.from_mnemonic(os.environ.get('CREATOR_MN'))
bidder = Account.from_mnemonic(os.environ.get('BUYER_MN'))
seller = Account.from_mnemonic(os.environ.get('SELLER_MN'))

storeAppID = int(os.environ.get("STORE_APP_ID"))
# close_bidding(client, 693, storeAppID, creator)
print(f"storeAppID: {storeAppID}")

print(f"Creator address: {creator.get_address()}")
print(f"Bidder address: {bidder.get_address()}")
print(f"Seller Address: {seller.get_address()}")

storeAppID: 2185
Creator address: 4TT75274EBUAF46CITUL6HQQ4C4D3GO7GEOVRZSQZ35VXSGRVHJ376GD64
Bidder address: X7PMCPPRD2T35ZZRTPETYK53G5YWBKUNONVDY52WODP3N3YM7NGEF5HGWU
Seller Address: Y4QR7LETUXJQVSPRKOJ5VFDR2GRO3MTSLJ4CLSIPJ5YN33Z3Z7HDT6JLYM


In [4]:
# def create_dummy_asset(sender: Account, total: int, decimals: int, asset_name: str, unit_name: str):
#     txn = transaction.AssetConfigTxn(
#         sender=sender.get_address(),
#         sp=client.suggested_params(),
#         total=total,
#         decimals=decimals,
#         asset_name=asset_name,
#         unit_name=unit_name,
#         default_frozen=False,
#         strict_empty_address_check=False,
#     )
#     signed_txn = txn.sign(sender.get_private_key())

#     client.send_transaction(signed_txn)

#     response = wait_for_confirmation(client, signed_txn.get_txid())
#     assert response.asset_index is not None and response.asset_index > 0
#     return response.asset_index

# asset_id = create_dummy_asset(seller, 1, 0, "example token", "AVT")

asset_id = int(os.environ.get("TOKEN_ID"))
print(f"The token id is: {asset_id}")

The token id is: 499


In [5]:
start_time = int(time()) + 10
end_time = start_time + 10
price = 1_000_000  # 1 Algo

"Alice is creating auction smart contract that lasts 30 seconds to auction off token..."

In [6]:
storeApp = StoringPool(client=client, creator=creator)
storeApp.create_app()
storeAppID = storeApp.app_id

# storeAppID = int(os.environ.get("STORE_APP_ID"))
# storeApp.app_id = storeAppID
# print(f"Store App ID: {storeAppID}")

storeAppAddress = get_application_address(storeAppID)
print(f"Store App Address: {storeAppAddress}")


Waiting for confirmation...
Transaction CVBQ665JIUOHTGNWHAXY6ZCFVIVBRKDIJF74QXB6IJN2YI3ZLAEA confirmed in round 112149.
Store App ID: 2439
Store App address: DZVXQ4GDM3LJCGLDWGLBQY7GUGRMH5ZMNVIA36BT6TOQMJCES2RN5ELI2Y
Waiting for confirmation...
Transaction AC3OVIFOY7AQSR5HBLHQPUJD3HEEPGVGZBLQESQVC77BIEIOGRDQ confirmed in round 112151.
Store App Address: DZVXQ4GDM3LJCGLDWGLBQY7GUGRMH5ZMNVIA36BT6TOQMJCES2RN5ELI2Y


In [7]:

appID = create_bidding_app(
    client=client,
    creator=creator,
    store_app_id=storeAppID
)
print(f"App ID: {appID}")
print(f"App Address: {get_application_address(appID)}")

Waiting for confirmation...
Transaction STFBKKD2BJ6NXYJZIAZLOAPM6OYYMIQPDHAX7LAVDARQTXJ3NPEQ confirmed in round 112153.
Waiting for confirmation...
Transaction XNKKXVHT3MY6IRTY5SCT6S3A36DUS25TCZUW2PUK26IGHLOV75EQ confirmed in round 112155.
App ID: 2441
App Address: SPWABFW3YTANSJI2TTW3T55N5AJ5J6QELXX3SBLWTXZIRICSDHJ3SBQYEI


Alice is setting up and funding token auction...

In [8]:
tradingAppID = create_trading_app(
    client=client,
    creator=creator,
    store_app_id=storeAppID
)
print(f"App ID: {appID}")
print(f"App Address: {get_application_address(appID)}")

Waiting for confirmation...
Transaction 7FJVH7ZUEJ3DLSNZL57MAFRLFLUQPBA24USCGO2RE6TX6EBBDVDQ confirmed in round 112157.
Waiting for confirmation...
Transaction VF7SEYDP4NZKN3MJB5OCDQI7JQ7X3OLWRAYAYAZTBIV3IMWGTD4A confirmed in round 112159.
App ID: 2441
App Address: SPWABFW3YTANSJI2TTW3T55N5AJ5J6QELXX3SBLWTXZIRICSDHJ3SBQYEI


In [9]:
storeApp.set_up(trade_app_id=tradingAppID, bid_app_id=appID, auction_app_id=123)

Waiting for confirmation...
Transaction BG6YUKA2HIKIYYS5U2DOALR3VGBW22OLL6J3TTEWYHOOEVKRVIAQ confirmed in round 112161.


In [10]:
sellerAlgosBefore = get_balances(client, seller.get_address())[0]

print("Alice's algo balance: ", sellerAlgosBefore, " algos")

Alice's algo balance:  20054796726320  algos


In [11]:
sleep(5)
actualAppBalancesBefore = get_balances(client, get_application_address(appID))
print("The smart contract now holds the following:", actualAppBalancesBefore)

The smart contract now holds the following: {0: 100000}


In [12]:
bidPrice = price
bidderAlgosBefore = get_balances(client, bidder.get_address())[0]
print("Carla wants to bid on token, her algo balance: ", bidderAlgosBefore, " algos")
print("Carla is placing bid for: ", bidPrice, " algos")

Carla wants to bid on token, her algo balance:  10025252649729  algos
Carla is placing bid for:  1000000  algos


In [13]:
trading_index = place_trade(client=client, app_id=tradingAppID, seller=seller, token_id=asset_id, token_amount=10, price=bidPrice, trading_index="")

print("Carla bid", trading_index)

store_app_id 2439
seller Y4QR7LETUXJQVSPRKOJ5VFDR2GRO3MTSLJ4CLSIPJ5YN33Z3Z7HDT6JLYM opt in app 2439
Waiting for confirmation...
Transaction CGJRNUSLZPXJZFC2WBUNNGT5ZZB52NCLRVKLWOOWQLSJPVRMJKFQ confirmed in round 112164.
Waiting for confirmation...
Transaction 2G4BIVHTYAXOZUEV4AK53LBCVLMDCISDYCA2Z6AUGS73BKUZ7BEA confirmed in round 112166.
new address: G5AM2YUER4JI4YNW32TDD3W7L4KRQFJMNET5LUXWYW5YFR5MEKYHRPMNCQ
new private_key: 0aO9aIJckHEl+R2BD+puFnAFQ6nEIcYIwyi6RiutPRs3QM1ihI8SjmG23qYx7t9fFRgVLGkn1dL2xbuCx6wisA==
new passphrase: kick walk crowd sight camera fragile since bulb usual popular danger accident april express census canyon boat around behave hill final hero supreme abandon hamster
Waiting for confirmation...
Transaction 4WSXGMZ3K2WIYKLB5M5XAYVGH52F37HW2NPH2ZQ23NL7SFLVK3NQ confirmed in round 112168.
Waiting for confirmation...
Transaction BD3IBHNBQATVK73UJQVHAPF6C3EUKAIBYBSKLM4VKYTAAY42S2NQ confirmed in round 112170.
Waiting for confirmation...
Transaction 4ZICFK3IMNWCUMXT6LYXW

In [14]:
accept_trade(client, tradingAppID, bidder, seller.get_address(), trading_index)

token_amount 10
price 1000000
Waiting for confirmation...
Transaction TBOK3JTX3HHNW3WRUDMKEGPE7UNZXNBOAZWOSJQHG4SITLHKPM2A confirmed in round 112176.
Waiting for confirmation...
Transaction 7PH36TH5KKFHCGQRMYN7FC4TMKNR7KTSNN3GN7MLDDLVW75TPNTQ confirmed in round 112178.


True

In [15]:
bid_index = place_bid(client=client, app_id=appID, token_id=asset_id, bidder=bidder, bid_amount=10, bid_price=bidPrice, bid_index="")

print("Carla bid", asset_id)

store_app_id 2439
new address: 33AFCREAJHZEAC4VEC2OKQFGBNCYFHJILESTWOE2EUQJSZDIRRGSGFFN64
new private_key: DToxBGhYAHRKToSW2Z5GZCpyhG0vFUtAPy8tx5fRIWjewFFEgEnyQAuVILTlQKYLRYKdKFklOziaJSCZZGiMTQ==
new passphrase: double shaft acoustic make absent other inch lounge crazy kiwi cash play tone genre kiwi benefit annual wonder nurse index convince inner donate about start
Waiting for confirmation...
Transaction 5GCPQXQIZIGKRDYBO5C6GTECL6PCUPJAKHRSU4NLESKANEKSAAYA confirmed in round 112180.
Waiting for confirmation...
Transaction JYYXZX5IG6Q5EKIZ4D2A4ZRBPNCNSWXA5B537MI3IDDEVKISGJZA confirmed in round 112182.
Waiting for confirmation...
Transaction EYUJQDZUY23L4ZG2GZ7NJSQODYEVBZRQD4HDIJLDWBXTWFXPPXNA confirmed in round 112184.
obj {'LOUBIMVHXMWMJNC4ZBKTIQTGNLF36EIXTEVBD7BUHDMIXKSMKQRNIPMQ6I': {'FOWXDCFQJKY3ISR7A5ZC2K5QEA5EYOJYCML55JWZVZYWWKQSUD6UFWIWKU': 1}, 'X7PMCPPRD2T35ZZRTPETYK53G5YWBKUNONVDY52WODP3N3YM7NGEF5HGWU': {'FOWXDCFQJKY3ISR7A5ZC2K5QEA5EYOJYCML55JWZVZYWWKQSUD6UFWIWKU': 1, 'OZPWYTPI

In [16]:
actualAppBalancesBefore = get_balances(client, get_application_address(appID))
print("The smart contract now holds the following:", actualAppBalancesBefore)

The smart contract now holds the following: {0: 1104000}


In [17]:
# account_info = client.account_info(bidder.get_address())  
# print(account_info.get('apps-local-state', []))
# print(is_opted_in_app(client, appID, bidder.get_address()))

# cancel_bid(client, appID, bidder, bid_index)

# print("Carla canceled bid", asset_id)

In [18]:
# bid_index = place_bid(client=client, app_id=appID, bidder=bidder, token_id=asset_id, bid_amount=10, bid_price=bidPrice, bid_index="")
# print(bid_index)
# bid_index = place_bid(client=client, app_id=appID, bidder=bidder, token_id=asset_id, bid_amount=20, bid_price=bidPrice+1_000_000, bid_index=bid_index)
# print(bid_index)
# bid_index = place_bid(client=client, app_id=appID, bidder=bidder, token_id=asset_id, bid_amount=15, bid_price=bidPrice+500_000, bid_index=bid_index)
# print(bid_index)


In [19]:
bidderAppLocalState = get_app_local_state(client, appID, bid_index)
print(bidderAppLocalState)

{b'B_ADDR': b'\xbf\xde\xc1=\xf1\x1e\xa7\xbe\xe71\x9b\xc9<+\xbb7q`\xaa\x8dsj<wVp\xdf\xb6\xef\x0c\xfbL', b'TA': 10, b'TK_ID': 499, b'TP': 1000000}


In [20]:
accept_bid(client, appID, seller, bidder.get_address(), bid_index)

token_amount 10
price 1000000
Waiting for confirmation...
Transaction YOX66UJ45ERCONPHW2IWUJHN7F7PAZJBLAWMGSHEPK5OT7SFVFMQ confirmed in round 112188.
Waiting for confirmation...
Transaction 7HGFH6IOCOLG5WKOJQ7YD3OHA26WQHSPVXROJ75TECQERZIALHMA confirmed in round 112190.


True

Alice is closing out the bidding contract....

In [21]:
sleep(5)

close_bidding(client, appID, creator, [asset_id])

b'assets' [499]
b'accounts' ['4TT75274EBUAF46CITUL6HQQ4C4D3GO7GEOVRZSQZ35VXSGRVHJ376GD64', 'UCAX3M5HS7PWWLB6HDCM2O7JE4HC3RGBPNWFRP4SYU62ZS5NSS2GFOKIOU']
Waiting for confirmation...
Transaction QCJF5CZ3EUU3KCFWGXVOWAUVEXDSOJ2SJS3PYU57EQ32K6HWUZHA confirmed in round 112193.


In [22]:
actualAppBalances = get_balances(client, get_application_address(appID))
expectedAppBalances = {0: 0}
print("The smart contract now holds the following:", actualAppBalances)

#assert actualAppBalances == expectedAppBalances

The smart contract now holds the following: {0: 335500, 499: 0}


In [23]:
bidderNftBalance = get_balances(client, bidder.get_address())[asset_id]

print("Carla's NFT balance:", bidderNftBalance, " for token ID: ", asset_id)

assert bidderNftBalance == 1

actualSellerBalances = get_balances(client, seller.get_address())
print("Alice's balances after auction: ", actualSellerBalances, " Algos")

actualBidderBalances = get_balances(client, bidder.get_address())
print("Carla's balances after auction: ", actualBidderBalances, " Algos")
# assert len(actualSellerBalances) == 2
# seller should receive the bid amount, minus the txn fee
assert actualSellerBalances[0] >= sellerAlgosBefore + bidPrice - 1_000
assert actualSellerBalances[asset_id] == 0

Carla's NFT balance: 155  for token ID:  499


AssertionError: 

In [ ]:
storeAppState = get_app_global_state(client, 1676)
print(storeAppState)
#print(encoding.encode_address(storeAppState[b"C"]))